### **TF-IDF란?**🔍

- TF-IDF는 문서 집합 내에서 특정 단어가 얼마나 중요한지를 평가하는 통계적 측정 도구입니다.
이 값은 두 가지 요소의 곱으로 계산됩니다:

> TF (Term Frequency) – 특정 단어가 해당 문서에서 얼마나 자주 등장하는지

> IDF (Inverse Document Frequency) – 해당 단어가 전체 문서 집합에서 얼마나 희귀한지

- 예를 들어, "this", "what", "if" 같은 단어는 모든 문서에 흔하게 등장하므로 TF가 높아도 IDF가 낮아져 TF-IDF 값이 낮습니다.
반면, "Bug" 같은 단어가 특정 문서에만 자주 등장하면 높은 TF-IDF 점수를 갖고, 해당 문서의 주요 주제를 나타낼 수 있습니다 (예: 신뢰성 관련 문서).

- 수식으로 표현된 TF-IDF 계산법: *TF-IDF(𝑡,𝑑)=TF(𝑡,𝑑)×IDF(𝑡)*



**Scikit-learn 라이브러리에 내장된 TF-IDF 알고리즘을 사용하여, 1987년부터 2019년까지 NeurIPS 학회에서 발표된 논문들로부터 상위 10개 키워드를 추출할 해봅시다** 😀

### **데이터 불러오기**

- 아래 셀을 실행시켜주세요

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 데이터 파일 이름 print
import pandas as pd
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/euron'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/drive/MyDrive/euron/All_English_Stopwords.txt
/content/drive/MyDrive/euron/kaggle_authors.csv
/content/drive/MyDrive/euron/kaggle_papers.csv
/content/drive/MyDrive/euron/Week10_복습과제_이름.ipynb


In [3]:
# Constants
# 위의 print된 내용에 따라 아래 path를 수정해주세요
PUNCTUATION = """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""
TOP_K_KEYWORDS = 10
STOPWORD_PATH = "/content/drive/MyDrive/euron/All_English_Stopwords.txt"
PAPERS_PATH = "/content/drive/MyDrive/euron/kaggle_papers.csv"

In [4]:
# 필요한 라이브러리 임포트
import re, os, string
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

### **키워드 추출을 위한 함수 정의** ⚓

#### **get stopwords list 함수 정의**
- 주어진 파일에서 불용어 목록을 읽어와 공백 제거 및 중복 제거 후 리스트 형태로 반환하는 함수입니다

- 아래 빈칸을 완성해주세요

In [5]:
def get_stopwords_list(stop_file_path):
    """load stop words """
     # 지정한 경로의 파일을 UTF-8 인코딩으로 읽기 모드 열기
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        # 파일의 모든 줄을 리스트로 읽어오기
        stopwords = f.readlines()
        # 각 줄에서 양쪽 공백을 제거하고 set으로 중복 제거하기
        stop_set = set(word.strip() for word in stopwords)
        # frozenset으로 다시 중복 제거하고 리스트로 변환 후 반환하기
        return list(frozenset(stop_set))

#### **clean text 함수 정의**
-  텍스트 데이터를 소문자로 변환, 문장부호 제거, 공백 정리하여 모델 학습이나 분석에 적합한 형태로 정제(cleaning) 함수입니다

- 아래 빈칸을 완성해주세요

In [6]:
def clean_text(text):
    # 모든 문자를 소문자로 변환하기
    text = text.lower()
    # 문장부호 제거하기 (PUNCTUATION에 정의된 모든 기호 제거)
    text = text.translate(str.maketrans('','',string.punctuation))
    # 공백 문자 및 줄바꿈을 하나의 공백으로 치환
    text = re.sub(r'\s+', ' ', text).strip()
    return text

#### **sort coo 함수 정의**
- 희소행령 (coo_matrix)를 TF-IDF 점수가 높은 단어부터 정렬하는 함수입니다.
- 아래 빈칸을 완성해주세요

In [7]:
def sort_coo(coo_matrix):
    # (단어 인덱스, 해당 인덱스의 TF-IDF 값) 튜플 생성하기
    tuples = list(zip(coo_matrix.col, coo_matrix.data))
    # TF-IDF 점수 기준으로 내림차순 정렬 (점수 같을 경우 인덱스 기준으로)
    return sorted(tuples, key=lambda x: (-x[1], x[0]))

#### **extract topn from vector 함수 정의**
- TF-IDF 벡터에서 상위 n개의 단어와 점수를  {단어: 점수} 형태로 추출하는 함수입니다.
- 아래 빈칸을 완성해주세요

In [8]:
# TF-IDF 벡터에서 상위 topn개의 단어와 점수를 추출하기
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    # 상위 topn 개 항목만 사용하기
    sorted_items = sorted_items[:topn]

    score_vals = [] # TF-IDF 점수 저장용 리스트
    feature_vals = [] # 단어 저장용 리스트

    # (단어 인덱스, TF-IDF 점수)로부터 단어와 점수를 추출하기
    for idx, score in sorted_items:
        score_vals.append(round(score, 3))  # 점수 반올림하여 저장
        feature_vals.append(feature_names[idx]) # 인덱스로 단어 찾아 저장

    # 단어: 점수 형태의 딕셔너리 생성하기
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]] = score_vals[idx]

    return results

#### **get keywords 함수 정의**
- 문서 하나에서 TF-IDF 기반으로 가장 중요한 단어 K개를 추출하는 함수입니다.
- 아래 빈칸을 완성해주세요

In [9]:
def get_keywords(vectorizer, feature_names, doc):
    """Return top k keywords from a doc using TF-IDF method"""

    # 주어진 문서에 대해 TF-IDF 벡터 생성하기
    tf_idf_vector = vectorizer.transform([doc])

    # TF-IDF 점수를 기준으로 내림차순 정렬하기
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    # 상위 TOP_K_KEYWORDS 개 단어와 점수 추출하기
    keywords=extract_topn_from_vector(feature_names, sorted_items, topn=TOP_K_KEYWORDS)
    # 키워드(단어)만 리스트로 반환하기
    return list(keywords.keys())

### **데이터에 적용하기** 🌻
- 우리의 데이터에 적용해봅시다!
- 아래 셀을 실행시켜주세요

In [10]:
data = pd.read_csv(PAPERS_PATH)
data.head()

,source_id,year,title,abstract,full_text
0,27,1987,Bit-Serial Neural Networks,NaN,573 \n\nBIT - SERIAL NEURAL NETWORKS \n\nAlan...
1,63,1987,Connectivity Versus Entropy,NaN,1 \n\nCONNECTIVITY VERSUS ENTROPY \n\nYaser S...
2,60,1987,The Hopfield Model with Multi-Level Neurons,NaN,278 \n\nTHE HOPFIELD MODEL WITH MUL TI-LEVEL N...
3,59,1987,How Neural Nets Work,NaN,442 \n\nAlan Lapedes \nRobert Farber \n\nThe...
4,69,1987,Spatial Organization of Neural Networks: A Pro...,NaN,740 \n\nSPATIAL ORGANIZATION OF NEURAL NEn...


In [11]:
data.dropna(subset=['full_text'], inplace=True)

In [12]:
# 위에서 정의한 함수로 data를 정제해줍니다
data['full_text'] = data['full_text'].apply(clean_text)

In [13]:
# 정제된 데이터를 확인해봅시다
data.head()

,source_id,year,title,abstract,full_text
0,27,1987,Bit-Serial Neural Networks,NaN,573 bit serial neural networks alan f murray a...
1,63,1987,Connectivity Versus Entropy,NaN,1 connectivity versus entropy yaser s abumosta...
2,60,1987,The Hopfield Model with Multi-Level Neurons,NaN,278 the hopfield model with mul tilevel neuron...
3,59,1987,How Neural Nets Work,NaN,442 alan lapedes robert farber theoretical div...
4,69,1987,Spatial Organization of Neural Networks: A Pro...,NaN,740 spatial organization of neural nenorks a p...


In [14]:
# 키워드를 추출하기 위해 full text를 list 형태로 corpora(말뭉치)에 적용해줍시다
corpora = data['full_text'].to_list()

In [15]:
# TF-IDF을 이용한 키워드 추출 ✅
# 불용어(stop words) 리스트 불러오기
stopwords=get_stopwords_list(STOPWORD_PATH)

In [17]:
# 불용어를 포함하여 TF-IDF Vectorizer 초기화하기
# smooth_idf=True는 log(IDF) 계산 시 1을 더해 0으로 나누는 것 방지해주세요!
vectorizer = TfidfVectorizer(stop_words=stopwords, smooth_idf=True)

# 말뭉치(corpora)로부터 단어 사전(vocabulary) 생성하기
# 앞의 10개 문서는 테스트용으로 제외하고 학습에 사용하기
vectorizer.fit(corpora[10:])

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['come', 'vis', 'viser', 'visest'] not in stop_words.
  warnings.warn(


TfidfVectorizer(stop_words=['ain', 'hadst', 'stopped', 'outs', 'latterest',
                            'yond', 'since', 'abover', 'sometimes', 'uponed',
                            'tilled', 'whosoever', 'username', 'inwarder',
                            'woulding', 'downs', 'how', 'hither', 'ups', 'till',
                            'because', 'doing', 'four', 'everybody', 'my',
                            'get', 'hoo', 'whereby', 'frae', 'provides', ...])

In [18]:
# 생성된 단어 사전을 feature_names에 저장하기
feature_names = vectorizer.get_feature_names_out()

In [19]:
result = []
# 앞의 10개 문서를 대상으로 반복하기
for doc in corpora[0:10]:
    df = {} # 하나의 문서 결과를 담을 딕셔너리 생성하기
    df['full_text'] = doc # 원본 문서 저장하기
    df['top_keywords'] = get_keywords(vectorizer, feature_names, doc) ## 상위 키워드 추출 후 저장하기
    result.append(df)

In [20]:
final = pd.DataFrame(result)
final

,full_text,top_keywords
0,573 bit serial neural networks alan f murray a...,"[synaptic, bit, vlsi, activation, state, array..."
1,1 connectivity versus entropy yaser s abumosta...,"[v2, h2k, 2n, en2, environment, neuron, h2, ed..."
2,278 the hopfield model with mul tilevel neuron...,"[qnn, neurons, hopfields, neuron, hopfield, ca..."
3,442 alan lapedes robert farber theoretical div...,"[bumps, net, bump, eqn, layer, hidden, output,..."
4,740 spatial organization of neural nenorks a p...,"[queueing, network, stimulations, cells, nodes..."
5,775 a neuralnetwork solution to the concentrat...,"[sites, neurons, assignment, subarray, site, a..."
6,642 learning by st ate recurrence detecfion br...,"[recurrence, aseace, failure, state, pole, ase..."
7,554 stability results for neural networks a n ...,"[equilibrium, stability, attraction, subsystem..."
8,804 introduction to a system for implementing ...,"[processors, processor, simd, connections, tim..."
9,474 optimiza non with artificial neural networ...,"[dipole, settling, eq, comer, dt, method, extr..."
